In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Token generation in cmd

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from io import BytesIO
import zipfile
import xml.etree.ElementTree as ET

API_TOKEN = '24bb102a-87a2-4b95-8eff-24e29eb94217'
BASE_URL = 'https://web-api.tp.entsoe.eu/api'
control_area_domain = '10Y1001A1001A83F'
DOCUMENT_TYPE = 'A85'

def fetch_data_for_period(start_str, end_str, max_retries=5):
    params = {
        'documentType': DOCUMENT_TYPE,
        'controlArea_Domain': control_area_domain,
        'periodStart': start_str,
        'periodEnd': end_str,
        'securityToken': API_TOKEN,
    }
    retry_delay = 5
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt+1}: fetching {start_str} to {end_str}")
            resp = requests.get(BASE_URL, params=params, timeout=30)
            resp.raise_for_status()
            content = resp.content
            # detect zip
            if 'application/zip' in resp.headers.get('Content-Type', '') or content.startswith(b'PK'):
                z = zipfile.ZipFile(BytesIO(content))
                xml_filename = z.namelist()[0]
                xml_content = z.read(xml_filename)
                return xml_content
            else:
                return content
        except requests.exceptions.Timeout:
            print("Timeout, retrying...")
            time.sleep(retry_delay)
            retry_delay *= 2
        except Exception as e:
            print("Error in fetch:", e)
            break
    return None

def parse_balancing_A85(xml_content):
    root = ET.fromstring(xml_content)

    # detect namespace
    nsuri = ''
    if root.tag.startswith('{'):
        nsuri = root.tag.split('}')[0].strip('{')
    ns = {'ns': nsuri} if nsuri else {}

    print("Root:", root.tag, "Namespace:", nsuri)

    records = []
    # find TimeSeries elements
    ts_path = './/ns:TimeSeries' if nsuri else './/TimeSeries'
    for ts in root.findall(ts_path, ns):
        # inside each TimeSeries, find Period
        period_path = 'ns:Period' if nsuri else 'Period'
        for period in ts.findall(period_path, ns):
            # timeInterval start/end
            ti = period.find('ns:timeInterval', ns) if nsuri else period.find('timeInterval')
            start = ti.find('ns:start', ns).text if ti is not None and ti.find('ns:start', ns) is not None else None
            end = ti.find('ns:end', ns).text if ti is not None and ti.find('ns:end', ns) is not None else None

            # iterate through Point elements
            point_path = 'ns:Point' if nsuri else 'Point'
            for point in period.findall(point_path, ns):
                pos_el = point.find('ns:position', ns) if nsuri else point.find('position')
                # here use imbalance_Price.amount
                val_el = point.find('ns:imbalance_Price.amount', ns) if nsuri else point.find('imbalance_Price.amount')
                cat_el = point.find('ns:imbalance_Price.category', ns) if nsuri else point.find('imbalance_Price.category')

                if pos_el is None or val_el is None:
                    continue

                rec = {
                    'start': start,
                    'end': end,
                    'position': int(pos_el.text),
                    'imbalance_price': float(val_el.text),
                }
                if cat_el is not None:
                    rec['price_category'] = cat_el.text
                records.append(rec)

    print("Parsed records:", len(records))
    return records

def daterange(start, end):
    for n in range(int((end - start).days)):
        yield start + timedelta(n)

def yyyymmddHHMM(dt):
    return dt.strftime('%Y%m%d%H%M')

# Main
start_date = datetime(2025, 9, 15, 0, 0)
end_date = datetime(2025, 9, 30, 23, 59)

all_recs = []
for d in daterange(start_date, end_date + timedelta(days=1)):
    day0 = d.replace(hour=0, minute=0)
    day1 = day0 + timedelta(days=1)
    s = yyyymmddHHMM(day0)
    e = yyyymmddHHMM(day1)
    xml = fetch_data_for_period(s, e)
    if xml:
        print(f"Fetched {s}-{e}, size {len(xml)}")
        recs = parse_balancing_A85(xml)
        all_recs.extend(recs)
    else:
        print("No xml")

df = pd.DataFrame(all_recs)
print("Total rows in df:", len(df))
print(df.head())

df.to_excel('entsoe_imbalance_prices.xlsx', index=False)
print("Saved to entsoe_imbalance_prices.xlsx")



Attempt 1: fetching 202509150000 to 202509160000
Fetched 202509150000-202509160000, size 35359
Root: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records: 192
Attempt 1: fetching 202509160000 to 202509170000
Fetched 202509160000-202509170000, size 35365
Root: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records: 192
Attempt 1: fetching 202509170000 to 202509180000
Fetched 202509170000-202509180000, size 35349
Root: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records: 192
Attempt 1: fetching 202509180000 to 202509190000
Fetched 202509180000-202509190000, size 35395
Root: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec623

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
import time
from io import BytesIO
import zipfile
import xml.etree.ElementTree as ET

API_TOKEN = 'INTRODUCE YOUR TOKEN HERE'
BASE_URL = 'https://web-api.tp.entsoe.eu/api'
control_area_domain = '10Y1001A1001A83F'
DOCUMENT_TYPE = 'A85'  # Imbalance Prices (Balancing)

def fetch_data_for_period(start_str, end_str, max_retries=5):
    params = {
        'documentType': DOCUMENT_TYPE,
        'controlArea_Domain': control_area_domain,
        'periodStart': start_str,
        'periodEnd': end_str,
        'securityToken': API_TOKEN,
    }
    retry_delay = 5
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt+1}: fetching {start_str} to {end_str}")
            resp = requests.get(BASE_URL, params=params, timeout=30)
            resp.raise_for_status()
            content = resp.content
            # If it's a ZIP archive, unzip
            if 'application/zip' in resp.headers.get('Content-Type', '') or content.startswith(b'PK'):
                zf = zipfile.ZipFile(BytesIO(content))
                xml_filename = zf.namelist()[0]
                xml_content = zf.read(xml_filename)
                return xml_content
            else:
                return content
        except requests.exceptions.Timeout:
            print(f"Timeout on attempt {attempt+1}, retrying after {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2
        except Exception as e:
            print("Error fetching data:", e)
            break
    print("Failed to fetch after retries.")
    return None

def parse_balancing_A85(xml_content):
    root = ET.fromstring(xml_content)
    # Determine namespace
    nsuri = ''
    if root.tag.startswith('{'):
        nsuri = root.tag.split('}')[0].strip('{')
    ns = {'ns': nsuri} if nsuri else {}

    print("Root tag:", root.tag, "Namespace:", nsuri)

    records = []
    # Use TimeSeries elements
    ts_path = './/ns:TimeSeries' if nsuri else './/TimeSeries'
    for ts in root.findall(ts_path, ns):
        # Inside each TimeSeries, find Period
        period_path = 'ns:Period' if nsuri else 'Period'
        for period in ts.findall(period_path, ns):
            # Extract timeInterval start / end
            ti = period.find('ns:timeInterval', ns) if nsuri else period.find('timeInterval')
            start = None
            end = None
            if ti is not None:
                st = ti.find('ns:start', ns) if nsuri else ti.find('start')
                en = ti.find('ns:end', ns) if nsuri else ti.find('end')
                if st is not None:
                    start = st.text
                if en is not None:
                    end = en.text

            # Now iterate through Point elements
            point_path = 'ns:Point' if nsuri else 'Point'
            for point in period.findall(point_path, ns):
                pos_el = point.find('ns:position', ns) if nsuri else point.find('position')
                val_el = point.find('ns:imbalance_Price.amount', ns) if nsuri else point.find('imbalance_Price.amount')
                cat_el = point.find('ns:imbalance_Price.category', ns) if nsuri else point.find('imbalance_Price.category')

                if pos_el is None or val_el is None:
                    continue

                rec = {
                    'start': start,
                    'end': end,
                    'position': int(pos_el.text),
                    'imbalance_price': float(val_el.text),
                }
                if cat_el is not None and cat_el.text is not None:
                    rec['price_category'] = cat_el.text
                records.append(rec)

    print("Parsed records count:", len(records))
    return records

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def yyyymmddHHMM(dt):
    return dt.strftime('%Y%m%d%H%M')

def compute_timestamp(start_str, position):
    """
    Given ISO start_str (e.g. '2025-09-15T00:00Z') and a 1‑based position,
    return a datetime offset by (position ‑ 1) * 15 minutes.
    Adjust the interval (15 min) if your data uses a different granularity.
    """
    dt0 = parser.isoparse(start_str)
    # Typically intervals are 15 minutes each; adjust multiplier if needed
    offset = timedelta(minutes=15 * (position - 1))
    return dt0 + offset

def main():
    # Define your date range
    start_date = datetime(2025, 9, 15, 0, 0)
    end_date = datetime(2025, 9, 30, 23, 59)

    all_recs = []
    for d in daterange(start_date, end_date + timedelta(days=1)):
        day0 = d.replace(hour=0, minute=0)
        day1 = day0 + timedelta(days=1)
        s = yyyymmddHHMM(day0)
        e = yyyymmddHHMM(day1)
        xml = fetch_data_for_period(s, e)
        if xml:
            print(f"Fetched data {s} to {e}, size {len(xml)}")
            recs = parse_balancing_A85(xml)
            all_recs.extend(recs)
        else:
            print(f"No data for interval {s}-{e}")

    df = pd.DataFrame(all_recs)
    print("Total rows in df:", len(df))
    print(df.head())

    if df.empty:
        print("WARNING: DataFrame is empty — no parsed records.")
    else:
        # Compute and format timestamp
        df['timestamp'] = df.apply(lambda row: compute_timestamp(row['start'], row['position']), axis=1)
        df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Optionally, keep only desired columns
        df = df[['timestamp', 'imbalance_price', 'price_category']]

        # Save to Excel
        output_fname = 'entsoe_imbalance_prices.xlsx'
        df.to_excel(output_fname, index=False)
        print(f"Saved to {output_fname}")

if __name__ == "__main__":
    main()



Attempt 1: fetching 202509150000 to 202509160000
Fetched data 202509150000 to 202509160000, size 35359
Root tag: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records count: 192
Attempt 1: fetching 202509160000 to 202509170000
Fetched data 202509160000 to 202509170000, size 35365
Root tag: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records count: 192
Attempt 1: fetching 202509170000 to 202509180000
Fetched data 202509170000 to 202509180000, size 35349
Root tag: {urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4}Balancing_MarketDocument Namespace: urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:4
Parsed records count: 192
Attempt 1: fetching 202509180000 to 202509190000
Fetched data 202509180000 to 202509190000, size 35395
Root tag: {urn:iec62325.351:tc57wg16:451-6:ba

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dateutil import parser
import time
from io import BytesIO
import zipfile
import xml.etree.ElementTree as ET

API_TOKEN = 'INTRODUCE YOUR TOKEN HERE'
BASE_URL = 'https://web-api.tp.entsoe.eu/api'
control_area_domain = '10Y1001A1001A83F'  # Germany/Luxembourg
DOCUMENT_TYPE = 'A85'  # Imbalance Prices

def fetch_data_for_period(start_str, end_str, max_retries=5):
    params = {
        'documentType': DOCUMENT_TYPE,
        'controlArea_Domain': control_area_domain,
        'periodStart': start_str,
        'periodEnd': end_str,
        'securityToken': API_TOKEN,
    }
    retry_delay = 5
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt+1}: fetching {start_str} to {end_str}")
            resp = requests.get(BASE_URL, params=params, timeout=30)
            resp.raise_for_status()
            content = resp.content
            if 'application/zip' in resp.headers.get('Content-Type', '') or content.startswith(b'PK'):
                zf = zipfile.ZipFile(BytesIO(content))
                xml_filename = zf.namelist()[0]
                xml_content = zf.read(xml_filename)
                return xml_content
            else:
                return content
        except requests.exceptions.Timeout:
            print(f"Timeout on attempt {attempt+1}, retrying after {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2
        except Exception as e:
            print("Error fetching data:", e)
            break
    print("Failed to fetch after retries.")
    return None

def parse_balancing_A85(xml_content):
    root = ET.fromstring(xml_content)
    nsuri = ''
    if root.tag.startswith('{'):
        nsuri = root.tag.split('}')[0].strip('{')
    ns = {'ns': nsuri} if nsuri else {}

    records = []
    ts_path = './/ns:TimeSeries' if nsuri else './/TimeSeries'
    for ts in root.findall(ts_path, ns):
        period_path = 'ns:Period' if nsuri else 'Period'
        for period in ts.findall(period_path, ns):
            ti = period.find('ns:timeInterval', ns) if nsuri else period.find('timeInterval')
            start = None
            if ti is not None:
                st = ti.find('ns:start', ns) if nsuri else ti.find('start')
                if st is not None:
                    start = st.text

            point_path = 'ns:Point' if nsuri else 'Point'
            for point in period.findall(point_path, ns):
                pos_el = point.find('ns:position', ns) if nsuri else point.find('position')
                val_el = point.find('ns:imbalance_Price.amount', ns) if nsuri else point.find('imbalance_Price.amount')
                cat_el = point.find('ns:imbalance_Price.category', ns) if nsuri else point.find('imbalance_Price.category')

                if pos_el is None or val_el is None or cat_el is None:
                    continue

                rec = {
                    'start': start,
                    'position': int(pos_el.text),
                    'price': float(val_el.text),
                    'category': cat_el.text.strip()
                }
                records.append(rec)
    print("Parsed records count:", len(records))
    return records

def compute_timestamp(start_str, position):
    dt0 = parser.isoparse(start_str)
    return dt0 + timedelta(minutes=15 * (position - 1))

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def yyyymmddHHMM(dt):
    return dt.strftime('%Y%m%d%H%M')

def main():
    start_date = datetime(2025, 9, 15, 0, 0)
    end_date = datetime(2025, 10, 1, 0, 0)

    all_recs = []
    for d in daterange(start_date, end_date):
        day0 = d.replace(hour=0, minute=0)
        day1 = day0 + timedelta(days=1)
        s = yyyymmddHHMM(day0)
        e = yyyymmddHHMM(day1)
        xml = fetch_data_for_period(s, e)
        if xml:
            print(f"Fetched data {s} to {e}, size {len(xml)}")
            recs = parse_balancing_A85(xml)
            all_recs.extend(recs)

    if not all_recs:
        print("No records found.")
        return

    raw_df = pd.DataFrame(all_recs)
    print("Total rows in raw df:", len(raw_df))

    raw_df['timestamp'] = raw_df.apply(lambda row: compute_timestamp(row['start'], row['position']), axis=1)
    raw_df = raw_df[['timestamp', 'price', 'category']]

    # ✅ Show actual categories found
    print("Categories found in data:", raw_df['category'].unique())

    # ✅ Pivot by category
    pivot_df = raw_df.pivot_table(
        index='timestamp',
        columns='category',
        values='price',
        aggfunc='first'
    ).reset_index()

    # ✅ Rename based on actual found codes
    col_map = {
        'A04': 'imbalance_price',
        'A05': 'downward_price',
        'A06': 'upward_price'
    }
    pivot_df.rename(columns=col_map, inplace=True)

    # ✅ Ensure all 3 columns exist
    for col in ['imbalance_price', 'upward_price', 'downward_price']:
        if col not in pivot_df:
            pivot_df[col] = None

    pivot_df.sort_values(by='timestamp', inplace=True)
    pivot_df['timestamp'] = pivot_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

    # ✅ Show preview
    print("\nFinal DataFrame preview:")
    print(pivot_df.head(20))

    # ✅ Save to Excel
    pivot_df.to_excel('entsoe_prices_combined.xlsx', index=False)
    print("✅ Saved to entsoe_prices_combined.xlsx")

if __name__ == "__main__":
    main()


Attempt 1: fetching 202509150000 to 202509160000
Fetched data 202509150000 to 202509160000, size 35359
Parsed records count: 192
Attempt 1: fetching 202509160000 to 202509170000
Fetched data 202509160000 to 202509170000, size 35365
Parsed records count: 192
Attempt 1: fetching 202509170000 to 202509180000
Fetched data 202509170000 to 202509180000, size 35349
Parsed records count: 192
Attempt 1: fetching 202509180000 to 202509190000
Fetched data 202509180000 to 202509190000, size 35395
Parsed records count: 192
Attempt 1: fetching 202509190000 to 202509200000
Fetched data 202509190000 to 202509200000, size 35381
Parsed records count: 192
Attempt 1: fetching 202509200000 to 202509210000
Fetched data 202509200000 to 202509210000, size 35337
Parsed records count: 192
Attempt 1: fetching 202509210000 to 202509220000
Fetched data 202509210000 to 202509220000, size 35373
Parsed records count: 192
Attempt 1: fetching 202509220000 to 202509230000
Fetched data 202509220000 to 202509230000, size 